# Funções de Perda

As funções de perda são responsáveis por retornar um valor numérico de comparação entre a saída do processamento de um modelo (rede neural) e a saída esperada para um determinado conjunto de entrada.

Essas funções precisam seguir alguns padrões de organização para sua utilização na cadeia de processamento (treinamento), integradas ao gráfico computacional gerado dinamicamente.

A biblioteca Pytorch já disponibiliza alguns tipos de função de perda, com [documentação](https://pytorch.org/docs/stable/nn.html#loss-functions) para expor os argumentos e aplicações de cada e [código-fonte](https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html) para detalhar a implementação e facilitar a extensão.

A partir do padrão destas funções pré-definidas é possível criar funções customizadas, a fim de atender problemas mais complexos e específicos.

Segue abaixo uma descrição de cada uma das funções de perda disponíveis no módulo torch.nn:

In [1]:
import os
import pkgutil

if pkgutil.find_loader('torch') is None:
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if os.path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl

In [2]:
import torch

---
## L1Loss
---

Calcula o módulo da diferença entre dois vetores, elemento a elemento. Pode ser aplicada a média, a soma ou nenhuma operação de redução.

$$L = [l_1, l_2, ..., l_N]^T, l_n = |x_n - y_n|$$

### Argumentos

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [3]:
inp = torch.randn(3, 5)
trg = torch.randn(3, 5)

In [4]:
l1_loss = torch.nn.L1Loss()
l1_loss(inp, trg)

tensor(1.3259)

---
## MSELoss
---

Calcula o erro quadrático médio entre dois tensores, de qualquer dimensão, elemento a elemento. Pode ser aplicada a média, a soma ou nenhuma operação de redução.

$$L = [l_1, l_2, ..., l_N]^T, l_n = (x_n - y_n)^2$$

### Argumentos

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [5]:
inp = torch.randn(3, 5, 4)
trg = torch.randn(3, 5, 4)

In [6]:
mse_loss = torch.nn.MSELoss()
mse_loss(inp, trg)

tensor(1.6164)

---
## CrossEntropyLoss
---

Combinação sequencial das funções **torch.nn.LogSoftmax()** e **torch.nn.NLLLoss()**, recebendo para cada classe as pontuações calculadas e, como target, o índice da classe esperada (que teria a maior pontuação).

Se não for incluída a lista de pesos como argumento de instanciação, este é o cálculo realizado:

$$L(x, class) = -\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}\right)
               = -x[class] + \log\left(\sum_j \exp(x[j])\right)$$

Se for incluída, o cálculo passa a ser esse:

$$L(x, class) = weight[class] \left(-x[class] + \log\left(\sum_j \exp(x[j])\right)\right)$$

### Argumentos

- **weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual ao número de classes

- **ignore_index**
 - Tipo: int
 - Demanda: opcional
 - Valor: índice do elemento a ser descartado no cálculo do gradiente

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [7]:
inp = torch.randn(3, 5)
trg = torch.empty(3, dtype=torch.long).random_(5)

In [8]:
ce_loss = torch.nn.CrossEntropyLoss()
ce_loss(inp, trg)

tensor(2.1989)

---
## NLLLoss
---

Cálculo da verossimilhança logarítmica negativa, para problemas de classificação.

É esperado que as entradas sejam probabilidades logarítmicas, com dimensão dada pelo número de classes. Os rótulos devem ser os índices das classes esperadas.

$$L = [l_1, l_2, ..., l_N]^T, \quad l_n = - w_{y_n} x_{n,y_n}, \quad
w_{c} = \text{weight}[c] \cdot \mathbb{1}\{c \not= \text{ignore_index}\}$$

### Argumentos

- **weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual ao número de classes

- **ignore_index**
 - Tipo: int
 - Demanda: opcional
 - Valor: índice do elemento a ser descartado no cálculo do gradiente

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [9]:
inp = torch.randn(3, 5)
trg = torch.empty(3, dtype=torch.long).random_(5)

In [10]:
nll_loss = torch.nn.NLLLoss()
nll_loss(inp, trg)

tensor(0.8225)

---
## PoissonNLLLoss
---

Cálculo da verossimilhança logarítmica negativa com distribuição de Poisson para o *target*.

$$target \sim Poisson(input)$$

$$l(input, target) = input - target * \log(input) + \log(target!)$$

### Argumentos

- **log_input**
 - Tipo: bool
 - Demanda: opcional
 - Valor padrão: True
 - Valor: Se *True*, a perda é calculada com a fórmula

$$exp(input) − target ∗ input$$

      Caso contrário, com a fórmula

$$input − target ∗ log(input+eps)$$

- **full**
 - Tipo: bool
 - Demanda: opcional
 - Valor padrão: False
 - Valor: Adiciona o termo de aproximação Stirling ao cálculo, resultado na fórmula

$$target * \log(target) - target + 0.5 * \log(2 * \pi * target)$$

- **eps**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: 1e-08
 - Valor: Pequena quantidade para evitar computar $log(0)$ quando *log_input* for *False*

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [11]:
inp = torch.randn(3, 5)
trg = torch.randn(3, 5)

In [12]:
pnll_loss = torch.nn.PoissonNLLLoss()
pnll_loss(inp, trg)

tensor(1.4339)

---
## KLDivLoss
---

Calcula a perda baseada na divergência Kullback-Leibler, que é uma medida de distância para distribuições contínuas, normalmente utilizada para realizar regressões diretas no espaço de distribuições contínuas (amostradas discretamente).

É esperado que as entradas sejam probabilidades logarítmicas, sem restrição de dimensão. Os rótulos devem ser probabilidades (sem a aplicação de logarítmo).

$$L = [l_1, l_2, ..., l_N]^T, \quad l_n = y_n \cdot \left( \log y_n - x_n \right)$$

### Argumentos

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [13]:
inp = torch.randn(3, 5)
trg = torch.randn(3, 5)

In [14]:
kld_loss = torch.nn.KLDivLoss()
kld_loss(inp, trg)

tensor(-0.1115)

---
## BCELoss
---

Calcula a Entropia Cruzada Binária, sendo o tensor de rótulos composto de números entre 0 e 1.

$$L = [l_1, l_2, ..., l_N]^T, \quad l_n = - w_n \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right]$$

### Argumentos

- **weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual ao tamanho do batch (N)

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

### Exemplo

In [15]:
inp = torch.empty(3).random_(2)
trg = torch.empty(3).random_(2)

In [16]:
bce_loss = torch.nn.BCELoss()
bce_loss(inp, trg)

tensor(9.2103)

---
## BCEWithLogitsLoss
---

Combinação da função Sigmoid com a função BCELoss, sendo o tensor de rótulos composto de números entre 0 e 1.

$$L = [l_1, l_2, ..., l_N]^T, \quad l_n = - w_n \left[ t_n \cdot \log \sigma(x_n)
+ (1 - t_n) \cdot \log (1 - \sigma(x_n)) \right]$$

Com o argumento **pos_weight** é possível balancear *precision* X *recall*, favorecendo este se o peso $p_n$ for maior que 1 e favorecendo aquele caso contrário.

$$L = [l_1, l_2, ..., l_N]^T, \quad l_n = - w_n \left[p_n \cdot t_n \cdot \log \sigma(x_n)
+ (1 - t_n) \cdot \log (1 - \sigma(x_n)) \right]$$

### Argumentos

- **weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual ao tamanho do batch (N)

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

- **pos_weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual número de classes

---
## MarginRankingLoss
---

Calcula elemento a elemento a diferença entre os tensores 1-D *x1* e *x2*, com ajuste do sinal pelo tensor *y* (composto de valores '-1' e '1', de mesma dimensão que os outros tensores).

$$l(x, y) = \max(0, -y * (x1 - x2) + margin)$$

### Argumentos

- **margin**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: 0
 - Valor: Offset para cálculo da perda

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## HingeEmbeddingLoss
---

Calcula a similaridade/dissimilaridade entre 2 entradas, por exemplo utilizando a distância pareada L1. A fórmula é escolhida pelo valor do tensor *y* (composto de valores '-1' e '1', de mesma dimensão que a entrada).

$$L = [l_1, l_2, ..., l_N]^T, \quad l_n = \begin{cases}
    x_n,\; \text{if}\; y_n = 1,\\
    \max \{0, margin - x_n\},\; \text{if}\; y_n = -1
\end{cases}$$

### Argumentos

- **margin**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: 1
 - Valor: Offset para cálculo da perda

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## MultiLabelMarginLoss
---

Calcula uma função de perda baseada em margem para multi-classificação de múltiplas classes, entre um tensor de entrada 2-D e um tensor de rótulos 2-D (com os índices das classes, com mesmo tamanho da entrada). Isso permite que diferentes amostras tenham um número variável de rótulos de classe.

$$l(x, y) = \sum_{ij}\frac{\max(0, 1 - (x[y[j]] - x[i]))}{\text{x.size}(0)} \\
where \; i = 0 \; to \; x.size(0), \quad j = 0 \; to \; y.size(0), \quad y[j] \; \ge \; 0, \quad i \; \neq \; y[j]$$

### Argumentos

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## SmoothL1Loss
---

Calcula uma versão suavizada da perda L1, igual a esta função se o valor absoluto da diferença entre elementos for maior ou igual a 1, ou a partir do quadrado da diferença se a mesma for menor que 1.

$$l(x, y) = \frac{1}{n} \sum_{i} z_{i} \; , \quad z_{i} =
\begin{cases}
0.5 (x_i - y_i)^2, \; \text{if} \; |x_i - y_i| < 1 \\
|x_i - y_i| - 0.5, \; \text{otherwise}
\end{cases}$$

### Argumentos

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## SoftMarginLoss
---

Calcula a perda logística para uma classificação de 2 classes, com tensor de entrada e tensor de rótulos contendo valores '-1' e '1'.

$$l(x, y) = \sum_i \frac{\log(1 + \exp(-y[i]*x[i]))}{\text{x.nelement}()}$$

### Argumentos

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## MultiLabelSoftMarginLoss
---

Calcula uma perda baseada na Entropia Máxima de um problema multi-rótulo de um contra todos.

$$l(x, y) = - \sum_i y[i] * \log((1 + \exp(-x[i]))^{-1})
                 + (1-y[i]) * \log\left(\frac{\exp(-x[i])}{(1 + \exp(-x[i]))}\right) \\
where \; i = 0 \; to \; x.nElement()-1, \quad y[i] \; in \; \{0,1\}$$

### Argumentos

- **weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual ao número de classes

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## CosineEmbeddingLoss
---

Calcula a similaridade/dissimilaridade entre 2 tensores de entrada baseado na distância cossenoidal. A fórmula é escolhida pelo valor do tensor *y* (composto de valores '-1' e '1', de mesma dimensão que a entrada).

$$l(x, y) = \begin{cases}
    1 - \cos(x_1, x_2), \quad \text{if} \; y == 1 \\
    \max(0, \cos(x_1, x_2) - \text{margin}), \quad \text{if} \; y == -1
\end{cases}$$

### Argumentos

- **margin**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: 0
 - Valor: Offset entre -1 e 1, sendo sugerido entre 0 e 0,5

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## MultiMarginLoss
---

Calcula uma função de perda baseada em margem para múltiplas classes, entre um tensor de entrada 2-D e um tensor de rótulos 1-D (com os índices das classes, de tamanho entre 0 e x.size(1)).

$$l(x, y) = \frac{\sum_i \max(0, (margin - x[y] + x[i])^p)}{\text{x.size}(0)} \\
where \; i = 0 \; to \; x.size(0), \quad i \; \neq \; y$$

Se for passado o argumento **weight**, o cálculo é alterado para

$$l(x, y) = \frac{\sum_i \max(0, w[y] * (margin - x[y] + x[i])^p)}{\text{x.size}(0)} \\
where \; i = 0 \; to \; x.size(0), \quad i \; \neq \; y$$

### Argumentos

- **p**
 - Tipo: int
 - Demanda: opcional
 - Valor padrão: 1
 - Valor: Apenas os valores 1 e 2 são suportados

- **margin**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: 1
 - Valor: Offset para cálculo da perda

- **weight**
 - Tipo: Tensor
 - Demanda: opcional
 - Tamanho: Dimensão única com número de elementos igual ao número de classes

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor

---
## TripletMarginLoss
---

Calcula uma medida de similaridade relativa entre amostras de 3 tensores de entrada. O trio para o cálculo são os valores âncora, exemplos positivos e exemplos negativos (*a*, *p* e *n*, respectivamente).

$$l(a, p, n) = \max \{d(a_i, p_i) - d(a_i, n_i) + {\rm margin}, 0\} \\
where \; d(x_i, y_i) = \left\lVert {\bf x}_i - {\bf y}_i \right\rVert_p$$

### Argumentos

- **margin**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: 1
 - Valor: Offset para cálculo da perda, maior que 0

- **p**
 - Tipo: int
 - Demanda: opcional
 - Valor padrão: 2
 - Valor: O grau da norma para a distância entre pares

- **swap**
 - Tipo: float
 - Demanda: opcional
 - Valor padrão: False
 - Valor: Explicado no paper [*Learning local feature descriptors with triplets and shallow convolutional neural networks*](http://www.bmva.org/bmvc/2016/papers/paper119/paper119.pdf) por V. Balntas et al.

- **reduction**
 - Tipo: string
 - Demanda: opcional
 - Valor padrão: ‘elementwise_mean’
 - Valores possíveis:

   ‘none’: nenhum efeito sobre a saída;

   ‘sum’: o vetor de saída é somado;

   ‘elementwise_mean’: a soma do vetor de saída é dividido pelo número de elementos do vetor